In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/updated-athletics-data/Collected Data - Sheet1.csv


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/updated-athletics-data/Collected Data - Sheet1.csv


In [3]:
df = pd.read_csv('/kaggle/input/updated-athletics-data/Collected Data - Sheet1.csv')
df.head

<bound method NDFrame.head of                                               Reviews    Categories
0   If you have any additional thoughts, comments ...  Fan Services
1   Make sure there is enough seats for students a...  Fan Services
2   I am extremely disappointed that you chose NOT...  Fan Services
3   I cannot see parts of the game with the 3 larg...  Fan Services
4   Free things for students - food, giveaways, et...  Fan Services
..                                                ...           ...
75             I thought the crowd was tremendous!!!!  Fan Services
76                             Underwhelming overall.  Fan Services
77  PA Announcer is too loud and it's shocking aft...  Fan Services
78  Felt there should be more commentary. It is fe...  Fan Services
79  Low price of tickets and no parking fee is suc...     Ticketing

[80 rows x 2 columns]>

In [4]:
df['Categories'].value_counts()

Categories
Fan Services    50
Ticketing       14
Parking          9
Concessions      7
Name: count, dtype: int64

In [5]:
# Split into features (reviews) and labels (categories)
X = df['Reviews']
Y = df['Categories']
len(X)

80

In [6]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=22)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report 

In [8]:
# Define pipelines for different models
pipeMNB = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')), ('clf', MultinomialNB())])
pipeCNB = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,3))), ('clf', ComplementNB())])
pipeSVC = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,3))), ('clf', LinearSVC())])

In [9]:
print(df.isnull().sum())

Reviews       0
Categories    0
dtype: int64


In [10]:
df['Reviews'] = df['Reviews'].fillna('')

In [11]:
df['Reviews'] = df['Reviews'].astype(str)

In [12]:
print(X_train.isnull().sum())  # Check for remaining missing values
print(X_train.dtypes)          # Check data types

0
object


In [13]:
pipeMNB.fit(X_train, Y_train)
predictMNB = pipeMNB.predict(X_test)
print(f"MNB Accuracy: {accuracy_score(Y_test, predictMNB):.2f}")

pipeCNB.fit(X_train, Y_train)
predictCNB = pipeCNB.predict(X_test)
print(f"CNB Accuracy: {accuracy_score(Y_test, predictCNB):.2f}")

pipeSVC.fit(X_train, Y_train)
predictSVC = pipeSVC.predict(X_test)
print(f"SVC Accuracy: {accuracy_score(Y_test, predictSVC):.2f}")

MNB Accuracy: 0.56
CNB Accuracy: 0.62
SVC Accuracy: 0.56


In [14]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Encode the target labels
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

# XGBoost pipeline
pipeXGB = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 2), stop_words = 'english')),
                    ('clf', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))])

# Train the model
pipeXGB.fit(X_train, Y_train_encoded)
predictXGB = pipeXGB.predict(X_test)

# Decode the predictions back to original labels
predictXGB_decoded = label_encoder.inverse_transform(predictXGB)

# Print accuracy
print(f"XGB Accuracy: {accuracy_score(Y_test, predictXGB_decoded):.2f}")


XGB Accuracy: 0.88


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2 

# Logistic Regression pipeline
pipeLR = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3), stop_words='english')),
                   ('select', SelectKBest(chi2, k=1000)),
                   ('clf', LogisticRegression(max_iter=1000))])

pipeLR.fit(X_train, Y_train)
predictLR = pipeLR.predict(X_test)
print(f"LR Accuracy: {accuracy_score(Y_test, predictLR):.2f}")

LR Accuracy: 0.56


In [16]:
from sklearn.ensemble import RandomForestClassifier
 
    

# Random Forest pipeline
pipeRF = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3), stop_words='english')),
                   ('clf', RandomForestClassifier())])

pipeRF.fit(X_train, Y_train)
predictRF = pipeRF.predict(X_test)
print(f"RF Accuracy: {accuracy_score(Y_test, predictRF):.2f}")

RF Accuracy: 0.62
